In [36]:
import numpy as np
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from sqlalchemy import create_engine
import psycopg2
import os
import plotly.figure_factory as ff
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

In [37]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = 5432,
    db = 'congress'
))

In [38]:
myquery = '''
SELECT directordername, bioguideid
FROM members
ORDER BY lastname
'''

members = pd.read_sql_query(myquery, con=engine)
member_list = [{'label': x, 'value': y} for x, y in zip(members['directordername'], members['bioguideid'])]
member_list

[{'label': 'Alma S. Adams', 'value': 'A000370'},
 {'label': 'Robert B. Aderholt', 'value': 'A000055'},
 {'label': 'Pete Aguilar', 'value': 'A000371'},
 {'label': 'Mark Alford', 'value': 'A000379'},
 {'label': 'Rick W. Allen', 'value': 'A000372'},
 {'label': 'Colin Z. Allred', 'value': 'A000376'},
 {'label': 'Mark E. Amodei', 'value': 'A000369'},
 {'label': 'Kelly Armstrong', 'value': 'A000377'},
 {'label': 'Jodey C. Arrington', 'value': 'A000375'},
 {'label': 'Jake Auchincloss', 'value': 'A000148'},
 {'label': 'Brian Babin', 'value': 'B001291'},
 {'label': 'Don Bacon', 'value': 'B001298'},
 {'label': 'James R. Baird', 'value': 'B001307'},
 {'label': 'Troy Balderson', 'value': 'B001306'},
 {'label': 'Tammy Baldwin', 'value': 'B001230'},
 {'label': 'Becca Balint', 'value': 'B001318'},
 {'label': 'Jim Banks', 'value': 'B001299'},
 {'label': 'Andy Barr', 'value': 'B001282'},
 {'label': 'Nanette Diaz Barragan', 'value': 'B001300'},
 {'label': 'John Barrasso', 'value': 'B001261'},
 {'label':

In [39]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [40]:
mymarkdown = '''

Congressional elections are usually all about the big national issues. However, these Representatives and Senators work for local districts and states. Not all issues are the same everywhere. The purpose of this dashboard is to collect public data from these sources:

* [Official API for the U.S. Congres](https://api.congress.gov/)

* [Voteview](https://voteview.com)

* [Open Secrets](https://www.opensecrets.org/open-data/api)

* ProPublica's Data on Bills


'''

In [44]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        #Stuff on top

        html.H1("Know Thy Representatives in Congress"),
        html.H2("Data collected from APIs from Congress.gov, Open Secrets, Voteview.com, and Propublica"),
        html.H3("DS6600: Data Engineering 1, UVA Data Science"),

        #Side bar

        html.Div([
            dcc.Markdown('Select a Representative or Senator:'),
            dcc.Dropdown(id = 'member',
                         options = member_list,
                         value = 'A000370'),
            dcc.Markdown(mymarkdown)
        ], style = {'width': '24%', 'float': 'left'}),

        #Main bar
        html.Div([
            dcc.Tabs([
                dcc.Tab(label = 'Biographical Info', children = [
                    #stuff for bio tab goes here
                    dcc.Graph(id = 'biotable')
                ]),
                dcc.Tab(label = 'Bill Sponsorship', children = [
                    #stuff for bill tab goes here
                ]),
                dcc.Tab(label = 'Voting and Ideology', children = [
                    #stuff for voting tab goes here
                ]),
                dcc.Tab(label = 'Donors and Financial Info', children = [
                    #stuff for donors tab goes here
                ])
            ])
        ], style = {'width': '74%', 'float': 'right'})
    ]
)

@app.callback([Output(component_id = 'biotable', component_property = 'figure')],
             [Input(component_id = 'member', component_property = 'value')])

def biotable(b):
    myquery = f'''
    SELECT directordername AS Name,
           party AS Party,
           state AS State,
           CAST(district AS int) AS District,
           birthyear AS Birthyear,
           addressinformation_officeaddress AS Address,
           CONCAT(addressinformation_city, ', ', addressinformation_district) AS City,
           addressinformation_zipcode AS zipcode,
           addressinformation_phonenumber
    FROM members
    WHERE bioguideid = '{b}'
    '''
    mydf = pd.read_sql_query(myquery, con=engine)
    mydf.columns = [x.capitalize() for x in mydf.columns]
    mydf = mydf.T.reset_index()
    mydf = mydf.rename({'index':'', 0: ''}, axis = 1)
    return [ff.create_table(mydf)]

if __name__ == "__main__":
    app.run_server(mode = 'external', host = "0.0.0.0", debug=True)
